In [ ]:
import pandas 
import uproot 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib
import urllib.request, json
from scipy.optimize import curve_fit

from tqdm.notebook import tqdm
import time
import sys
import multiprocessing

In [ ]:
RUN1 = True

datadir = "/icarus/data/users/gputnam/DMCP2023G/normdata/"

savedir = datadir + ("run1_" if RUN1 else "run2_")

dosave = False

In [ ]:
savedir

In [ ]:
if RUN1:
    # Run 1, OnBeam
    rootfiles = [
        "norminfo_Run1_Onbeam_majority.root",
        "norminfo_Run1_Onbeam_minbias.root"
    ]
else:
    # Run 2, OnBeam
    rootfiles = [
        "norminfo_Run2_Onbeam.root"
    ]

In [ ]:
fs = [uproot.open(datadir + f) for f in rootfiles]
dfts = [f["icarusnumi"]["TriggerInfo"].arrays(library="pd") for f in fs]

In [ ]:
dft = pd.concat(dfts, ignore_index=True, sort=False)


In [ ]:
dft["prev_trigger_timestamp_s"] = dft.prev_trigger_timestamp / 1_000_000_000
dft["trigger_timestamp_s"] = dft.trigger_timestamp / 1_000_000_000

In [ ]:
t0 = dft.trigger_timestamp_s.min()
t1 = dft.trigger_timestamp_s.max()

In [ ]:
# Chunk the request into 1-hour

qtimes = np.linspace(t0, t1, int((t1-t0) / (60*60)))
print(len(qtimes))

In [ ]:
def get_ifbeam_data(device, qtimes, ilo=None, ihi=None):
    times = []
    dat = []
    for i, (tlo, thi) in tqdm(enumerate(zip(qtimes[:-1], qtimes[1:])), total=len(qtimes)-1):
        url = "https://dbdata3vm.fnal.gov:9443/ifbeam/data//data?v=E:%s&e=e,a9&t0=%f&t1=%f&f=json" % (device, tlo, thi)
        
        success = False
        itry = 0
        while not success:
            try:
                with urllib.request.urlopen(url) as url:
                    data = json.load(url)
            except:
                if itry > 4:
                    raise
                itry += 1
                time.sleep(5)
                continue
                
            success = True

        times += [r["clock"]/1e3 for r in data["rows"]]
        dat += [[v["v"] for v in r["c"][1:]] for r in data["rows"]]
        
    if ilo and ihi:
        dat = [d[ilo:ihi] for d in dat]
            
                
    return times, np.squeeze(np.array(dat))

In [ ]:
# POT
pots = get_ifbeam_data("TRTGTD", qtimes)

In [ ]:
# Data for horn current
nslina = get_ifbeam_data("NSLINA", qtimes)
nslinb = get_ifbeam_data("NSLINB", qtimes)
nslinc = get_ifbeam_data("NSLINC", qtimes)
nslind = get_ifbeam_data("NSLIND", qtimes)

In [ ]:
# Data for beam spot
hptgt = get_ifbeam_data("HPTGT[]", qtimes)
hitgt = get_ifbeam_data("HITGT[]", qtimes)
vptgt = get_ifbeam_data("VPTGT[]", qtimes)
vitgt = get_ifbeam_data("VITGT[]", qtimes)

hp121 = get_ifbeam_data("HP121[]", qtimes)
vp121 = get_ifbeam_data("VP121[]", qtimes)

In [ ]:
# Beam width
mtgtds = get_ifbeam_data("MTGTDS[]", qtimes, ilo=103, ihi=199)

In [ ]:
# merge together nslin DB

nslindf = pd.DataFrame(data={"time": nslina[0], "a": nslina[1]})
nslindf = pd.merge_asof(nslindf, pd.DataFrame(data={"time": nslinb[0], "b": nslinb[1]}), 
                        on="time", tolerance=0.5, direction="nearest")
nslindf = pd.merge_asof(nslindf, pd.DataFrame(data={"time": nslinc[0], "c": nslinc[1]}), 
                        on="time", tolerance=0.5, direction="nearest")
nslindf = pd.merge_asof(nslindf, pd.DataFrame(data={"time": nslind[0], "d": nslind[1]}), 
                        on="time", tolerance=0.5, direction="nearest")

In [ ]:
# Compute current
horncurrent = ( ((nslindf.a - 0.01)/0.9951) + ((nslindf.b - (-0.14))/0.9957) + ((nslindf.c - (-0.05))/0.9965) + ((nslindf.d - (-0.07))/0.9945) )

In [ ]:
# bad index
# Bad spill in run 2
if ~RUN1:
    badindex = (np.array(hitgt[0][:-1]) != np.array(hptgt[0])).argmax()
    hitgt = np.delete(hitgt[0], badindex, axis=0), np.delete(hitgt[1], badindex, axis=0)
    vp121 = np.delete(vp121[0], badindex, axis=0), np.delete(vp121[1], badindex, axis=0)

In [ ]:
# Compute beam spot

mmPerFoot = 12.*2.54*10.

z_hp121 = -68.04458 * mmPerFoot
z_vp121 = -66.99283 * mmPerFoot
z_hptgt = -31.25508 * mmPerFoot
z_vptgt = -30.16533 * mmPerFoot
z_targ  = 0.

def extrapToLoc(var1, loc1, var2, loc2, loc3):
    slope = (var2-var1)/(loc2-loc1)
    proj = var1 + ((loc3-loc1)*slope)
    return proj;

if RUN1:
    x_nom121 = 0
    x_nomtgt = 0
    y_nom121 = -1.5
    y_nomtgt = -1.5
    x_nomTarg = 0.398
    y_nomTarg = -0.39
else:
    x_nom121 = 1.2
    x_nomtgt = 0.7
    y_nom121 = -0.4
    y_nomtgt = -0.94
    x_nomTarg = 0.03
    y_nomTarg = -0.59
    
xCorr = extrapToLoc(x_nom121,z_hp121,x_nomtgt,z_hptgt,z_targ) - x_nomTarg
yCorr = extrapToLoc(y_nom121,z_hp121,y_nomtgt,z_hptgt,z_targ) - y_nomTarg

ave_hp121 = np.ma.average(hp121[1], axis=-1)
ave_vp121 = np.ma.average(vp121[1], axis=-1)

ave_hptgt = np.ma.average(hptgt[1], weights=hitgt[1], axis=-1)
ave_vptgt = np.ma.average(vptgt[1], weights=vitgt[1], axis=-1)

extrap_hptgt = extrapToLoc(ave_hp121, z_hp121, ave_hptgt, z_hptgt, z_targ) - xCorr
extrap_vptgt = extrapToLoc(ave_vp121, z_vp121, ave_vptgt, z_vptgt, z_targ) - yCorr

In [ ]:
# Compute width
def gaus_constant(x, C, A, mu, sigma):
    return C + A*np.exp(-(x-mu)**2/(2*sigma**2))

xs = (np.array(list(range(48))) - 23.5)/2

def do_mtgtds_fit(M):
    i0 = 103
    h_chan = np.array(M[103-i0:151-i0])
    v_chan = np.array(M[151-i0:199-i0])
    
    try:
        popt_h, _ = curve_fit(gaus_constant, xs, -h_chan, p0=[np.mean(-h_chan), np.max(-h_chan), 0, 2], maxfev=10_000)
    except:
        popt_h = [np.nan]
    try:
        popt_v, _ = curve_fit(gaus_constant, xs, -v_chan, p0=[np.mean(-v_chan), np.max(-v_chan), 0, 2], maxfev=10_000)
    except:
        popt_v = [np.nan]
        
    return [popt_h[-1], popt_v[-1]]

def fit_iter(i):
    return mtgtds[0][i], do_mtgtds_fit(mtgtds[1][i, :])

dats = []
with multiprocessing.Pool(20) as pool:
    for dat in tqdm(pool.imap_unordered(fit_iter, range(mtgtds[1].shape[0])), total=mtgtds[1].shape[0]):
        dats.append(dat)
        
beamwidths = list(map(np.array, zip(*dats)))stay

In [ ]:
# Load data into dataframes
potdf = pd.DataFrame(data={"time": pots[0], "pot": pots[1]})
hcdf = pd.DataFrame(data={"time": nslina[0], "hc_time": nslina[0], "horn_current": horncurrent})
posdf = pd.DataFrame(data={"time": hptgt[0], "pos_time": hptgt[0], "hp121": ave_hp121, "vp121": ave_vp121, "hptgt": ave_hptgt, "vptgt": ave_vptgt, "extrap_hptgt" : extrap_hptgt, "extrap_vptgt": extrap_vptgt})
bwdf = pd.DataFrame(data={"time": beamwidths[0], "bw_time": beamwidths[0], "beam_width_h": beamwidths[1][:, 0], "beam_width_v": beamwidths[1][:, 1]}).sort_values("time").reset_index(drop=True)

In [ ]:
# Merge with timing information

# POT info is the base
beamdf = pd.merge_asof(potdf, hcdf, on="time", tolerance=0.5, direction="nearest")
beamdf = pd.merge_asof(beamdf, posdf, on="time", tolerance=0.5, direction="nearest")
beamdf = pd.merge_asof(beamdf, bwdf, on="time", tolerance=0.5, direction="nearest")

In [ ]:
# Validate beamdf 
assert(np.abs(beamdf.time - beamdf.hc_time).max() < 0.2)
assert(np.abs(beamdf.time - beamdf.pos_time).max() < 0.2)
assert(np.abs(beamdf.time - beamdf.bw_time).max() < 0.2)

In [ ]:
del beamdf["hc_time"]
del beamdf["pos_time"]
del beamdf["bw_time"]

In [ ]:
beamdf

In [ ]:
# Distributions

In [ ]:
_ = plt.hist(beamdf.pot, bins=np.linspace(-1, 70, 101))
plt.xlabel("POT [$\\times 10^{12}$]")
plt.axvline(2, color="red")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_pot.pdf")
    plt.savefig(savedir + "spill_pot.svg")

In [ ]:
_ = plt.hist(beamdf.horn_current, bins=np.linspace(-203, -195, 101))
plt.xlabel("Horn Current [mA]")
plt.axvline(-202, color="red")
plt.axvline(-196.4, color="red")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_horncurrent.pdf")
    plt.savefig(savedir + "spill_horncurrent.svg")

In [ ]:
_ = plt.hist(beamdf.extrap_hptgt, bins=np.linspace(-2, 2, 101))
plt.axvline(1, color="red")
plt.axvline(-1, color="red")
plt.xlabel("Beam Position X [mm]")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_beamposx.pdf")
    plt.savefig(savedir + "spill_beamposx.svg")

In [ ]:
_ = plt.hist(beamdf.extrap_vptgt, bins=np.linspace(-2, 2, 101))
plt.axvline(1, color="red")
plt.axvline(-1, color="red")
plt.xlabel("Beam Position Y [mm]")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_beamposy.pdf")
    plt.savefig(savedir + "spill_beamposy.svg")

In [ ]:
_ = plt.hist(beamdf.beam_width_h, bins=np.linspace(0.5, 4, 101))
plt.axvline(0.57, color="red")
plt.axvline(1.88, color="red")
plt.xlabel("Horizontal Beam Width [mm]")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_beamwidthh.pdf")
    plt.savefig(savedir + "spill_beamwidthh.svg")

In [ ]:
_ = plt.hist(beamdf.beam_width_v, bins=np.linspace(0.5, 4, 101))
plt.axvline(0.57, color="red")
plt.axvline(1.88, color="red")
plt.xlabel("Vertical Beam Width [mm]")
plt.tight_layout()
plt.ylabel("Spills")
if dosave: 
    plt.savefig(savedir + "spill_beamwidthv.pdf")
    plt.savefig(savedir + "spill_beamwidthv.svg")

In [ ]:
if dosave:
    beamdf.to_hdf(savedir + "beam.df", key="beam")